In [21]:
# Bag of words vs Tfidf

In [22]:
import pandas as pd
import numpy as np

data_url = 'https://raw.githubusercontent.com/Himanshu-1703/reddit-sentiment-analysis/refs/heads/main/data/reddit.csv'
df = pd.read_csv(data_url)

In [23]:
df

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1
...,...,...
37244,jesus,0
37245,kya bhai pure saal chutiya banaya modi aur jab...,1
37246,downvote karna tha par upvote hogaya,0
37247,haha nice,1


In [24]:
# import mlflow
# import dagshub

# dagshub.init(repo_owner='CodeNeuron58', repo_name='Insightube', mlflow=True)

In [25]:
# mlflow.set_experiment('Final - 1')

In [26]:
# from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
# vectorizer = TfidfVectorizer(max_features=max_features, ngram_range=ngram_range)
# X_train = train_data['clean_comment'].values
# y_train = train_data['category'].values
# X_train_tfidf = vectorizer.fit_transform(X_train)

In [28]:
df = df.dropna()
df = df.drop_duplicates()
df = df[df['clean_comment'].str.strip() != '']

In [29]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [30]:
train_df

,clean_comment,category
36510,found young modi guess which one,1
8959,holy shit heard the whole johnny gosch story e...,0
26360,porra tava amando ler isso poderia pelo menos ...,0
6306,mmw will have something witty add this thread ...,1
30267,have similar chart for call quality would lov...,0
...,...,...
17011,would just saying msm cant sensetionalize peo...,-1
6313,india will have counter strike team major atle...,1
11376,well think just like family member the same li...,0
867,this very good stuff thank you for taking the ...,1


In [31]:
import pandas as pd
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download required NLTK data
nltk.download('wordnet')
nltk.download('stopwords')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bipra\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bipra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [32]:
from nltk.stem import PorterStemmer

In [ ]:
def preprocess_comment(comment):
    """Apply preprocessing transformations to a comment."""
    # Convert to lowercase
    comment = comment.lower()

    # Remove trailing and leading whitespaces
    comment = comment.strip()

    # Remove newline characters
    comment = re.sub(r'\n', ' ', comment)

    # Remove non-alphanumeric characters, except punctuation
    comment = re.sub(r'[^A-Za-z0-9\s!?.,]', '', comment)

    # Remove stopwords but retain important ones for sentiment analysis
    stop_words = set(stopwords.words('english')) - {'not', 'but', 'however', 'no', 'yet'}
    comment = ' '.join([word for word in comment.split() if word not in stop_words])

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    comment = ' '.join([lemmatizer.lemmatize(word) for word in comment.split()])
    

    return comment

# Normalize the text in both train and test datasets
train_df['clean_comment'] = train_df['clean_comment'].apply(preprocess_comment)
test_df['clean_comment'] = test_df['clean_comment'].apply(preprocess_comment)

In [ ]:
train_df

,clean_comment,category
36510,found young modi guess one,1
8959,holi shit heard whole johnni gosch stori earli...,0
26360,porra tava amando ler isso poderia pelo meno d...,0
6306,mmw someth witti add thread diwali 2016,1
30267,similar chart call qualiti would love see low ...,0
...,...,...
17011,would say msm cant sensetion peopl hate respon...,-1
6313,india counter strike team major atleast one three,1
11376,well think like famili member line line line l...,0
867,good stuff thank take time post point rais dis...,1


In [ ]:
test_df

,clean_comment,category
3824,hunter vanguard armor incred dat blood splatter,1
13598,fuck cunt done feed flesh probabl show intoler...,-1
21341,happi one great thing happen recent,1
35986,sourc behind articl seem like propoganda edit ...,-1
2976,like guy mayb someday replac china india,0
...,...,...
10095,team standard pick one jordan matthew isaiah c...,0
7742,base apu,0
17216,ravish journalist india rest becom govern mout...,0
35477,let releas unemploy data report guess releas e...,1


In [ ]:
import pandas as pd
import os
import pickle
import yaml
import lightgbm as lgb
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:

max_features = 10000
ngram_range = (1, 3)
learning_rate = 0.09
max_depth = 20
n_estimators = 367

In [ ]:
vectorizer = TfidfVectorizer(max_features=max_features, ngram_range= ngram_range)
X_train = train_df['clean_comment'].values
y_train = train_df['category'].values
X_train_tfidf = vectorizer.fit_transform(X_train)

In [ ]:
model = lgb.LGBMClassifier(
    objective='multiclass',
    num_class=3,
    metric="multi_logloss",
    is_unbalance=True,
    class_weight="balanced",
    reg_alpha=0.1,  # L1 regularization
    reg_lambda=0.1,  # L2 regularization
    learning_rate=learning_rate,
    max_depth=max_depth,
    n_estimators=n_estimators
)
model.fit(X_train_tfidf, y_train)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.327301 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 135924
[LightGBM] [Info] Number of data points in the train set: 29434, number of used features: 4157
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further

,boosting_type,'gbdt'
,num_leaves,31
,max_depth,20
,learning_rate,0.09
,n_estimators,367
,subsample_for_bin,200000
,objective,'multiclass'
,class_weight,'balanced'
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

X_test = test_df['clean_comment'].values
y_test = test_df['category'].values
X_test_tfidf = vectorizer.transform(X_test)

y_pred = model.predict(X_test_tfidf)
report = classification_report(y_test, y_pred, output_dict=True)
cm = confusion_matrix(y_test, y_pred)
accuracy_score = accuracy_score(y_test, y_pred)

print("Classification Report:\n", report)
print("Confusion Matrix:\n", cm)
print("Accuracy Score:", accuracy_score)

c:\Campusx\Insightube\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Classification Report:
 {'-1': {'precision': 0.7913312693498452, 'recall': 0.7648114901256733, 'f1-score': 0.7778454047474133, 'support': 1671.0}, '0': {'precision': 0.8419232099619509, 'recall': 0.9408581368380363, 'f1-score': 0.8886454910551296, 'support': 2587.0}, '1': {'precision': 0.8980021030494216, 'recall': 0.8261851015801355, 'f1-score': 0.8605979173664763, 'support': 3101.0}, 'accuracy': 0.8525614893327897, 'macro avg': {'precision': 0.8437521941204059, 'recall': 0.8439515761812818, 'f1-score': 0.8423629377230064, 'support': 7359.0}, 'weighted avg': {'precision': 0.8540663699974745, 'recall': 0.8525614893327897, 'f1-score': 0.8516673051292282, 'support': 7359.0}}
Confusion Matrix:
 [[1278  176  217]
 [  79 2434   74]
 [ 258  281 2562]]
Accuracy Score: 0.8525614893327897
